In [1]:
import os
import random
from rdflib import Graph

In [2]:
benchmark_folder = '../benchmarks/DASHB/subgraphs/{pid}/'
patient_data_file = '../benchmarks/DASHB/patients/{pid}.ttl'

In [3]:
patient = Graph()
patient.parse(patient_data_file.format(pid='zplus_1'), format='turtle')

<Graph identifier=N59de15e9466f42b18439d8633c698199 (<class 'rdflib.graph.Graph'>)>

### Entity2Id.txt

In [4]:
entities = list(patient.subjects(predicate=None, object=None))
entities.extend(list(patient.objects(subject=None, predicate=None)))
unique_entities = list(set(entities))

print(f'Unique entities = {len(unique_entities)}')

Unique entities = 87


In [5]:
with open(os.path.join(benchmark_folder, 'entity2id.txt'), 'w') as relation_f:
    relation_f.write(str(len(unique_entities)) + '\n')
    
    for idx in range(len(unique_entities)):
        relation_f.write(f'{unique_entities[idx]}\t{idx}\n')

### Relation2Id.txt

In [6]:
relations = patient.predicates(subject=None, object=None)
unique_relations = list(set(relations))

print(f'Unique relations = {len(unique_relations)}')

Unique relations = 14


In [7]:
with open(os.path.join(benchmark_folder, 'relation2id.txt'), 'w') as relation_f:
    relation_f.write(str(len(unique_relations)) + '\n')
    
    for idx in range(len(unique_relations)):
        relation_f.write(f'{unique_relations[idx]}\t{idx}\n')

### Triples 

(e1, e2, rel) cf. [documentation](http://139.129.163.161/static/notes/data.html)

In [8]:
triples = list(patient.triples((None, None, None)))

print(f'Triples = {len(triples)}')

Triples = 171


In [9]:
split_ratio = [0.8, 0.15, 0.05]  # [train, test, validation]
slices = []

start = 0
for split in split_ratio:
    end = round(split * len(triples))
    slices.append((start, min(start + end, len(triples) - 1)))
    start += end
print(slices)

[(0, 137), (137, 163), (163, 170)]


In [10]:
validation = triples[slices[2][0]:slices[2][1]]

train, test, validation = [triples[slice_[0]:slice_[1]] for slice_ in slices]
print(len(train), len(test), len(validation))

137 26 7


In [11]:
with open(os.path.join(benchmark_folder, 'train2id.txt'), 'w') as train_f:
    train_f.write(str(len(train)) + '\n')
    
    for idx in range(len(train)):
        e1, rel, e2 = train[idx]
        e1_id = unique_entities.index(e1)
        e2_id = unique_entities.index(e2)
        rel_id = unique_relations.index(rel)
        
        train_f.write(f'{e1_id}\t{e2_id}\t{rel_id}\n')

In [12]:
with open(os.path.join(benchmark_folder, 'test2id.txt'), 'w') as test_f:
    test_f.write(str(len(test)) + '\n')
    
    for idx in range(len(test)):
        e1, rel, e2 = test[idx]
        e1_id = unique_entities.index(e1)
        e2_id = unique_entities.index(e2)
        rel_id = unique_relations.index(rel)
        
        test_f.write(f'{e1_id}\t{e2_id}\t{rel_id}\n')

In [13]:
with open(os.path.join(benchmark_folder, 'valid2id.txt'), 'w') as validation_f:
    validation_f.write(str(len(validation)) + '\n')
    
    for idx in range(len(validation)):
        e1, rel, e2 = validation[idx]
        e1_id = unique_entities.index(e1)
        e2_id = unique_entities.index(e2)
        rel_id = unique_relations.index(rel)
        
        validation_f.write(f'{e1_id}\t{e2_id}\t{rel_id}\n')